In [0]:
import torch
torch.cuda.get_device_name(0)

'Tesla T4'

In [0]:
!pip install transformers

     |████████████████████████████████| 573kB 21.1MB/s 
     |████████████████████████████████| 3.7MB 52.5MB/s 


In [0]:
from transformers import pipeline
# 情感分类
nlp = pipeline("sentiment-analysis")

print(nlp("I hate you"))
print(nlp("I love you"))


[{'label': 'NEGATIVE', 'score': 0.9991129}]
[{'label': 'POSITIVE', 'score': 0.99986565}]


### 下面是一个使用模型进行序列分类的示例，以确定两个序列是否是彼此的解释。该过程如下：

从checkpoint名称实例化一个tokenizer和一个模型。该模型被识别为一个BERT模型，并用存储在checkpoint中的权重加载它。

从这两句话中构建一个序列，使用正确的特定于模型的分隔符标记类型id和注意力掩码(encode()和encode_plus()处理这个问题)

将这个序列传递到模型中，以便将其分类到两个可用的类中的一个：0(不是解释)和1(是解释)

计算结果的softmax获取类的概率

打印结果

In [0]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification


tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")

classes = ["not paraphrase", "is paraphrase"]

sequence_0 = "The company HuggingFace is based in New York City"
sequence_1 = "Apples are especially bad for your health"
sequence_2 = "HuggingFace's headquarters are situated in Manhattan"

paraphrase = tokenizer.encode_plus(sequence_0, sequence_2, return_tensors="pt")
not_paraphrase = tokenizer.encode_plus(sequence_0, sequence_1, return_tensors="pt")

print(paraphrase)
print(not_paraphrase)


{'input_ids': tensor([[  101,  1109,  1419, 20164, 10932,  2271,  7954,  1110,  1359,  1107,
          1203,  1365,  1392,   102, 20164, 10932,  2271,  7954,   112,   188,
          3834,  1132,  3629,  1107,  6545,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])}
{'input_ids': tensor([[  101,  1109,  1419, 20164, 10932,  2271,  7954,  1110,  1359,  1107,
          1203,  1365,  1392,   102,  7302,  1116,  1132,  2108,  2213,  1111,
          1240,  2332,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [0]:
paraphrase_classification_logits = model(**paraphrase)[0]
not_paraphrase_classification_logits = model(**not_paraphrase)[0]

print(model(**paraphrase))
print(paraphrase_classification_logits)
print(not_paraphrase_classification_logits)

(tensor([[-0.3495,  1.9004]], grad_fn=<AddmmBackward>),)
tensor([[-0.3495,  1.9004]], grad_fn=<AddmmBackward>)
tensor([[ 0.5386, -2.2197]], grad_fn=<AddmmBackward>)


In [0]:
paraphrase_results = torch.softmax(paraphrase_classification_logits, dim=1).tolist()[0]
not_paraphrase_results = torch.softmax(not_paraphrase_classification_logits, dim=1).tolist()[0]

print(torch.softmax(paraphrase_classification_logits, dim=1))
print(torch.softmax(paraphrase_classification_logits, dim=1).tolist())
print(paraphrase_results)
print(not_paraphrase_results)

tensor([[0.0954, 0.9046]], grad_fn=<SoftmaxBackward>)
[[0.0953628346323967, 0.9046371579170227]]
[0.0953628346323967, 0.9046371579170227]
[0.94038325548172, 0.059616751968860626]


In [0]:
print("Should be paraphrase")
for i in range(len(classes)):
    print(f"{classes[i]}: {round(paraphrase_results[i] * 100)}%")

print("\nShould not be paraphrase")
for i in range(len(classes)):
    print(f"{classes[i]}: {round(not_paraphrase_results[i] * 100)}%")

Should be paraphrase
not paraphrase: 10%
is paraphrase: 90%

Should not be paraphrase
not paraphrase: 94%
is paraphrase: 6%
